In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Reload dataset
frames_metadata = pd.read_csv(".../balanced_frames_metadata.csv")
frames_metadata["label"] = frames_metadata["label"].astype(str)  # Convert labels to string

# Image Data Generator with Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,   # Rotate images randomly
    width_shift_range=0.2,  # Shift images left-right
    height_shift_range=0.2,  # Shift images up-down
    horizontal_flip=True  # Flip images
)

# Training Data Generator
train_gen = datagen.flow_from_dataframe(
    dataframe=frames_metadata,
    directory=".../extracted_frames/",
    x_col="frame_path",
    y_col="label",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Validation Data Generator
val_gen = datagen.flow_from_dataframe(
    dataframe=frames_metadata,
    directory=".../extracted_frames/",
    x_col="frame_path",
    y_col="label",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define CNN Model
image_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (fraud/non-fraud)
])

image_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
image_model.fit(train_gen, validation_data=val_gen, epochs=10)

# Save the trained CNN model
image_model.save(".../image_model.h5")

print("✅ Image Model Training Complete!")